In [ ]:
# file:///C:/Users/danih/Downloads/A%201012782908347.pdf Hierarchical Text CategorizationUsing Neural Networks MIGUEL E. RUIZ


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset
import json
import math
import random
import pandas as pd 
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download("wordnet")
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/ 

In [ ]:
label_columns = ['pornographic-content', 'violence', 'death', 'sexual-assault', 'abuse', 'blood',
                 'suicide', 'pregnancy', 'child-abuse', 'incest', 'underage', 'homophobia',
                 'self-harm', 'dying', 'kidnapping', 'mental-illness', 'dissection',
                 'eating-disorders', 'abduction', 'body-hatred', 'childbirth', 'racism',
                 'sexism', 'miscarriages', 'transphobia', 'abortion', 'fat-phobia',
                 'animal-death', 'ableism', 'classism', 'misogyny', 'animal-cruelty']

# Cleaning Text

In [ ]:
# Cleaning Document for Document:
# - html code
# - special characters
# - multiple spaces
# - lowercasing
# - non alphabetic characters
# - non english words
# - lemmatization

import pandas as pd
import json
import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk import pos_tag
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

english_words = set(wordnet.words())

# Define the label columns in the desired order

CREATE_DATA = False

def preprocess_function(text):
    # Remove irrelevant data and special characters
    text = BeautifulSoup(text, "html.parser").get_text(strip=True)
    text = re.sub(r'\xa0', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Convert text to lowercase
    text = text.lower()

    text = re.sub(r'(\w)\1{2,}', '', text) # Remove words with the same letter repeated four times or more

    # Tokenize the text into words
    words = word_tokenize(text)

    # Preprocess each word using list comprehensions
    words = [re.sub(r'[^a-zA-Z]', '', word) for word in words]  # Remove non-alphabetic characters

    # Filter out non-English words
    words = [word for word in words if word in english_words]

    # Perform POS tagging for lemmatization
    #tagged_words = pos_tag(words)

    # Lemmatize the word based on its POS tag
    #lemmatizer = WordNetLemmatizer()
    #words = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos_tag))
    #         for word, pos_tag in tagged_words if word]

    return ' '.join(words)

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

def clean_data(path):
    # Initialize empty lists to store the extracted data
    contexts = []
    work_ids = []
    label_values = {col: [] for col in label_columns}

    # Read the JSONL file and process the data
    with open(path, 'r') as file:
        for line_num, line in enumerate(tqdm(file)):
            data = json.loads(line)
            work_ids.append(data["work_id"])
            contexts.append(data["text"])

            for i, label_col in enumerate(label_columns):
                label_values[label_col].append(data['labels'][i])     

            #if line_num == 1000:
            #    break

    # Preprocess the text data in parallel using all available CPU cores
    with tqdm(total=len(contexts), desc="Preprocessing") as pbar:
        with ProcessPoolExecutor() as executor:
            contexts = list(executor.map(preprocess_function, contexts))
            pbar.update(len(contexts))

    # Create a dictionary with the extracted data
    data_dict = {'text': contexts, 'work_id': work_ids}
    data_dict.update(label_values)

    # Create a DataFrame from the dictionary
    df = pd.DataFrame(data_dict)

    #df.to_csv('cleaned_data.csv', index=False)
    #df = pd.read_csv('cleaned_data.csv')
    
    return df

if CREATE_DATA:
    train = clean_data("/kaggle/input/pan23trigger/pan23-trigger-detection/pan23-trigger-detection-train/works.jsonl")
    val = clean_data("/kaggle/input/pan23trigger/pan23-trigger-detection/pan23-trigger-detection-validation/works.jsonl")
else:
    train = pd.read_csv('/kaggle/input/tf-idf-valtrain-nolemmas-cleaned/train_cleaned_data.csv')    
    val = pd.read_csv('/kaggle/input/tf-idf-valtrain-nolemmas-cleaned/val_cleaned_data.csv')
    
train.to_csv('train_cleaned_data.csv', index=False)
val.to_csv('val_cleaned_data.csv', index=False)

train

# NA Filling

In [ ]:
train[train["text"].isna()]

In [ ]:
train = train.dropna()
train

# Data balancing methods

In [ ]:
import pandas as pd
from itertools import combinations
from collections import Counter
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

def undersample_class_combinations(df, label_columns):
    # Get the class combinations for each data entry
    class_combinations = df[label_columns].apply(tuple, axis=1)
    combination_counts = Counter(class_combinations)
    
    # Print the top 15 most common classes and their counts before undersampling
    most_common_classes_before = combination_counts.most_common(15)
    print("Top 15 Most Common Classes Before Undersampling:")
    for combination, count in most_common_classes_before:
        print(f"Class Combination: {combination}, Count: {count}")

    # Plot class combinations before undersampling
    plt.figure(figsize=(10, 12))
    plt.subplot(2, 1, 1)
    plt.bar(range(len(combination_counts)), list(combination_counts.values()))
    plt.title('Class Combinations Before Undersampling')
    plt.ylabel('Count (log scale)')
    plt.yscale('log') #symlog
    plt.xticks([])

    # Sort class combinations by count in descending order
    sorted_combinations = sorted(combination_counts.items(), key=lambda x: x[1], reverse=True)

    # Calculate the maximum allowed count for the majority class combinations
    max_count = sorted_combinations[0][1]
    second_max_count = sorted_combinations[1][1]
    max_count_limit = int(second_max_count * 2)

    # Create a dictionary to map class combinations to indices
    combination_indices = {}
    for i, cc in enumerate(class_combinations):
        if cc not in combination_indices:
            combination_indices[cc] = [i]
        else:
            combination_indices[cc].append(i)

    # Undersample the majority class combinations
    undersampled_indices = [random.sample(indices, max_count_limit) for _, indices in tqdm(combination_indices.items()) if len(indices) > max_count_limit]
    undersampled_indices = [idx for indices in undersampled_indices for idx in indices]

    # Create the balanced DataFrame
    df_balanced = df.iloc[undersampled_indices].copy()
    df_balanced.reset_index(drop=True, inplace=True)

    # Include entries from other classes
    for _, indices in tqdm(combination_indices.items()):
        if len(indices) <= max_count_limit:
            df_balanced = pd.concat([df_balanced, df.iloc[indices]])

    # Update the counts of class combinations after undersampling
    class_combinations_balanced = df_balanced[label_columns].apply(tuple, axis=1)
    combination_counts_balanced = Counter(class_combinations_balanced)

    # Plot class combinations after undersampling
    plt.subplot(2, 1, 2)
    plt.bar(range(len(combination_counts_balanced)), list(combination_counts_balanced.values()))
    plt.title('Class Combinations After Undersampling')
    plt.xlabel('Class Combinations')
    plt.ylabel('Count (log scale)')
    plt.yscale('log') #symlog

    plt.tight_layout()
    plt.show()
    
    # Print the top 15 most common classes and their counts after undersampling
    most_common_classes_after = combination_counts_balanced.most_common(15)
    print("\nTop 15 Most Common Classes After Undersampling:")
    for combination, count in most_common_classes_after:
        print(f"Class Combination: {combination}, Count: {count}")

    return df_balanced

#train = train.head(100000)
#train = undersample_class_combinations(train, label_columns)
#train

# Hierarchy

In [ ]:
g_abuse = ["abuse", "child-abuse", 'sexual-assault', 'underage']
g_discrimination = ['mental-illness', 'eating-disorders', 'transphobia', 'ableism', 'classism', 'misogyny', 'racism', 
                  'fat-phobia', 'homophobia', 'sexism', 'body-hatred']
g_aggresion = ["violence", 'death', 'dying', 'kidnapping', 'abduction']
g_medical = ['blood', "dissection"]
g_mental = ['self-harm', 'suicide']
g_sexual = ['pornographic-content', 'incest']
g_pregnancy = ['childbirth', "abortion", 'miscarriages', 'pregnancy']
g_animal = ['animal-death', 'animal-cruelty']

coarse_grained = ["g_abuse", "g_discrimination", "g_aggresion", "g_medical", "g_mental", "g_sexual", "g_pregnancy", "g_animal"]
coarse_grained_arrays = [g_abuse, g_discrimination, g_aggresion, g_medical, g_mental, g_sexual, g_pregnancy, g_animal]

In [ ]:
train["g_abuse"] = (train[g_abuse] == 1).any(axis=1).astype(int)
train["g_discrimination"] = (train[g_discrimination] == 1).any(axis=1).astype(int)
train["g_aggresion"] = (train[g_aggresion] == 1).any(axis=1).astype(int)
train["g_medical"] = (train[g_medical] == 1).any(axis=1).astype(int)
train["g_mental"] = (train[g_mental] == 1).any(axis=1).astype(int)
train["g_sexual"] = (train[g_sexual] == 1).any(axis=1).astype(int)
train["g_pregnancy"] = (train[g_pregnancy] == 1).any(axis=1).astype(int)
train["g_animal"] = (train[g_animal] == 1).any(axis=1).astype(int)

train

# Data Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, y_train = train[['text', 'work_id']], train[label_columns + coarse_grained]
X_val, y_val = val[['text', 'work_id']], val[label_columns]

del train, val

In [ ]:
weights = []
for label in label_columns:
    weight = len(y_train["pornographic-content"]) / y_train[label].sum()
    weights.append(weight)

print(weights)

# TF IDF Encoding

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1,1), max_features=50000)

tfidf_df = vectorizer.fit_transform(X_train["text"])
tfidf_df

# Train

In [ ]:
from sklearn.svm import SVC
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

test_vec = vectorizer.transform(X_val["text"])
test_vec

In [ ]:
coarse_grained_model = MultiOutputClassifier(LinearSVC()).fit(tfidf_df, y_train[coarse_grained])

coarse_grained_pred = coarse_grained_model.predict(test_vec)
coarse_grained_pred = pd.DataFrame(coarse_grained_pred, columns=coarse_grained)
coarse_grained_pred

In [ ]:
coarse_grained = ["g_abuse", "g_discrimination", "g_aggresion", "g_medical", "g_mental", "g_sexual", "g_pregnancy", "g_animal"]
coarse_grained_arrays = [g_abuse, g_discrimination, g_aggresion, g_medical, g_mental, g_sexual, g_pregnancy, g_animal]

for column, arrays in zip(coarse_grained, coarse_grained_arrays):
    print("In validation set coarse", column, "-", (y_val[arrays] == 1).any(axis=1).astype(int).sum(), ", predicted:", coarse_grained_pred[column].sum())

In [ ]:
fine_grained_models = []

for column, arrays in zip(coarse_grained, coarse_grained_arrays):
    mask = y_train[column] == 1
    model = MultiOutputClassifier(LinearSVC()).fit(tfidf_df[mask], y_train[mask][arrays])

    prediction_mask = coarse_grained_pred[column] == 1
    pred = model.predict(test_vec[prediction_mask])
    
    X_val[arrays] = 0
    X_val.loc[prediction_mask, arrays] = pred
    
    fine_grained_models.append(model)
    
    print(column, ", train data: ", len(y_train[mask]))
    for f_col in arrays:
        print("   >>>> ", f_col, "predicted", X_val[f_col].sum(), "total", y_val[f_col].sum())
        
X_val

In [ ]:
print(accuracy_score(y_val, X_val[label_columns]))
print(f1_score(y_val, X_val[label_columns], average='macro', zero_division=1))
print(f1_score(y_val, X_val[label_columns], average='micro', zero_division=1))

# Interpretation

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

In [ ]:
estimators = []
n_of_top_words = 250

for m in fine_grained_models:
    for e in m.estimators_:
        estimators.append(e)

flattened_fine_grained_labels = [item for sublist in coarse_grained_arrays for item in sublist]
most_important_words_list = [[] for _ in range(n_of_top_words)]

for name, estimator in zip(flattened_fine_grained_labels, estimators):    
    top_words = np.argsort(estimator.coef_[0])[::-1][0:n_of_top_words]
    
    print(name)
    
    for idx, value in enumerate(top_words):
        most_important_words_list[idx].append(feature_names[value])
        
        if idx < 10:
            print("  >>", feature_names[value])
    print("--------------------")

In [ ]:
with open('most_important_words.json', 'w') as json_file:
    json.dump(most_important_words_list, json_file)

# Reading the list of lists from the JSON file
with open('most_important_words.json', 'r') as json_file:
    loaded_list = json.load(json_file)

loaded_list